In [ ]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import files

files.upload()

{}

In [ ]:
mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download nodoubttome/skin-cancer9-classesisic --unzip

100% 786M/786M [00:35<00:00, 24.4MB/s]
100% 786M/786M [00:35<00:00, 23.3MB/s]


In [ ]:
import pandas as pd
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import time
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
sns.set_style('darkgrid')
import shutil
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import time
from tqdm import tqdm
from sklearn.metrics import f1_score
from IPython.display import YouTubeVideo
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199
print ('Modules loaded')
import pathlib
import pathlib
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import PIL
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam # - Works
import random
from glob import glob
import seaborn as sns
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import matplotlib.pyplot as plt
import matplotlib.image as img
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

Modules loaded


In [ ]:
! rm -rf /kaggle/working/data/

In [ ]:
data_dir_train = pathlib.Path("/content/Skin cancer ISIC The International Skin Imaging Collaboration/Train")
data_dir_test = pathlib.Path("/content/Skin cancer ISIC The International Skin Imaging Collaboration/Test")

In [ ]:
image_count_train = len(list(data_dir_train.glob('*/*.jpg')))
print(image_count_train)
image_count_test = len(list(data_dir_test.glob('*/*.jpg')))
print(image_count_test)

2287
128


In [ ]:
!find . -name ".ipynb_checkpoints" -type d -exec rm -rf {} +

In [ ]:
batch_size = 32
img_height = 256
img_width = 256
rnd_seed = 123
IMG_INPUT_SHAPE  = (180, 180, 3)
random.seed(rnd_seed)

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_train,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 2287 files belonging to 10 classes.
Using 1830 files for training.


In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_train,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 2287 files belonging to 10 classes.
Using 457 files for validation.


In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_test,
  validation_split=0.9,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 128 files belonging to 10 classes.
Using 115 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)

['Non-Skin Cancer', 'actinic keratosis', 'basal cell carcinoma', 'dermatofibroma', 'melanoma', 'nevus', 'pigmented benign keratosis', 'seborrheic keratosis', 'squamous cell carcinoma', 'vascular lesion']


In [ ]:
pip install Augmentor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
path_to_training_dataset = '/content/Skin cancer ISIC The International Skin Imaging Collaboration/Train/'
import Augmentor
for i in class_names:
    p = Augmentor.Pipeline(path_to_training_dataset + i, output_directory='/kaggle/working/data/'+i+'/output/')
    p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
    p.sample(1000) 


Initialised with 48 image(s) found.
Output directory set to /kaggle/working/data/Non-Skin Cancer/output/.

Processing <PIL.Image.Image image mode=RGB size=411x482 at 0x7F919C33B940>: 100%|██████████| 1000/1000 [01:17<00:00, 12.85 Samples/s]


Initialised with 114 image(s) found.
Output directory set to /kaggle/working/data/actinic keratosis/output/.

Processing <PIL.Image.Image image mode=RGB size=600x450 at 0x7F923023B400>: 100%|██████████| 1000/1000 [00:37<00:00, 26.57 Samples/s]


Initialised with 376 image(s) found.
Output directory set to /kaggle/working/data/basal cell carcinoma/output/.

...

Processing <PIL.Image.Image image mode=RGB size=1024x768 at 0x7F919C5C74F0>: 100%|██████████| 1000/1000 [01:26<00:00, 11.55 Samples/s]


Initialised with 181 image(s) found.
Output directory set to /kaggle/working/data/squamous cell carcinoma/output/.

Processing <PIL.Image.Image image mode=RGB size=600x450 at 0x7F923023B400>: 100%|██████████| 1000/1000 [00:37<00:00, 26.82 Samples/s]


Initialised with 139 image(s) found.
Output directory set to /kaggle/working/data/vascular lesion/output/.

Processing <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=600x450 at 0x7F919C5F2F80>: 100%|██████████| 1000/1000 [00:36<00:00, 27.45 Samples/s]


In [ ]:
output_dir = pathlib.Path('/kaggle/working/data/')
image_count_train = len(list(output_dir.glob('*/output/*.jpg')))
print(image_count_train)

10000


In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  output_dir,
  seed=123,
  validation_split = 0.2,
  subset = 'training',
  image_size=(img_height, img_width),
  batch_size=batch_size)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  output_dir,
  seed=123,
  validation_split = 0.2,
  subset = 'validation',
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 10000 files belonging to 10 classes.
Using 8000 files for training.
Found 10000 files belonging to 10 classes.
Using 2000 files for validation.


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
IMG_INPUT_SHAPE=(256,256,3)

In [ ]:
def get_model(tf_model):
    model = None
    if tf_model == "VGG16":
        model = tf.keras.applications.VGG16(include_top=False, weights="imagenet", input_tensor = tf.keras.Input(shape=IMG_INPUT_SHAPE))
    elif tf_model == "ResNet50":
        model = tf.keras.applications.ResNet50(include_top=False, weights="imagenet", input_tensor = tf.keras.Input(shape=IMG_INPUT_SHAPE))
    elif tf_model == "MobileNetV2":
        model = tf.keras.applications.MobileNetV2(include_top=False, weights="imagenet", input_tensor = tf.keras.Input(shape=IMG_INPUT_SHAPE))
    else:
        raise Exception('SUNXYZ', 'Model unknown')
    return_model = tf.keras.models.Sequential()
    return_model.add(model)
    return_model.add(tf.keras.layers.Flatten())
    return_model.add(tf.keras.layers.Dense(512))
    return_model.add(tf.keras.layers.Dense(256, activation='relu'))
    return_model.add(tf.keras.layers.Dense(64, activation='relu'))
    return_model.add(tf.keras.layers.Dense(10, activation='softmax'))
    return return_model

In [ ]:
BASIC_MODEL = tf.keras.models.Sequential()
BASIC_MODEL.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
BASIC_MODEL.add(tf.keras.layers.MaxPooling2D((2, 2)))
BASIC_MODEL.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
BASIC_MODEL.add(tf.keras.layers.MaxPooling2D((2, 2)))
BASIC_MODEL.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
BASIC_MODEL.add(tf.keras.layers.Flatten())
BASIC_MODEL.add(tf.keras.layers.Dense(256, activation='relu'))
BASIC_MODEL.add(tf.keras.layers.Dense(64, activation='relu'))
BASIC_MODEL.add(tf.keras.layers.Dense(10, activation='softmax'))

In [ ]:
VGG16_MODEL       = get_model("VGG16")
RESNET50_MODEL    = get_model("ResNet50")
MOBILENETv2_MODEL = get_model("MobileNetV2")

In [ ]:
VGG16_MODEL.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 32768)             0         
                                                                 
 dense_3 (Dense)             (None, 512)               16777728  
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dense_5 (Dense)             (None, 64)                16448     
                                                                 
 dense_6 (Dense)             (None, 10)                650       
                                                                 
Total params: 31,640,842
Trainable params: 31,640,842


In [ ]:
RESNET50_MODEL.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 flatten_2 (Flatten)         (None, 131072)            0         
                                                                 
 dense_7 (Dense)             (None, 512)               67109376  
                                                                 
 dense_8 (Dense)             (None, 256)               131328    
                                                                 
 dense_9 (Dense)             (None, 64)                16448     
                                                                 
 dense_10 (Dense)            (None, 10)                650       
                                                                 
Total params: 90,845,514
Trainable params: 90,792,394


In [ ]:
MOBILENETv2_MODEL.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 8, 8, 1280)       2257984   
 ional)                                                          
                                                                 
 flatten_3 (Flatten)         (None, 81920)             0         
                                                                 
 dense_11 (Dense)            (None, 512)               41943552  
                                                                 
 dense_12 (Dense)            (None, 256)               131328    
                                                                 
 dense_13 (Dense)            (None, 64)                16448     
                                                                 
 dense_14 (Dense)            (None, 10)                650       
                                                      

In [ ]:
def compile_and_fit(model):
    model.compile(optimizer='adam',
      loss=tf.losses.SparseCategoricalCrossentropy(),
      metrics=['accuracy'])
    history = model.fit(
      train_ds,
      validation_data=val_ds,
      epochs=10
    )
    return history

In [ ]:
VGG16_MODEL.save('vgg16_model.h5')

In [ ]:
RESNET50_HISTORY = compile_and_fit(RESNET50_MODEL)

Epoch 1/10
250/250 [==============================] - 181s 455ms/step - loss: 3.3531 - accuracy: 0.3235 - val_loss: 1.7693 - val_accuracy: 0.3445
Epoch 2/10
250/250 [==============================] - 111s 444ms/step - loss: 1.4512 - accuracy: 0.4555 - val_loss: 2.3033 - val_accuracy: 0.2990
Epoch 3/10
250/250 [==============================] - 111s 444ms/step - loss: 1.0806 - accuracy: 0.5954 - val_loss: 2.0161 - val_accuracy: 0.4205
Epoch 4/10
250/250 [==============================] - 111s 444ms/step - loss: 0.8202 - accuracy: 0.7075 - val_loss: 3.0250 - val_accuracy: 0.3280
Epoch 5/10
250/250 [==============================] - 111s 444ms/step - loss: 0.6795 - accuracy: 0.7551 - val_loss: 1.3614 - val_accuracy: 0.5770
Epoch 6/10
250/250 [==============================] - 111s 444ms/step - loss: 0.6035 - accuracy: 0.7934 - val_loss: 1.2790 - val_accuracy: 0.6295
Epoch 7/10
250/250 [==============================] - 109s 438ms/step - loss: 0.4474 - accuracy: 0.8361 - val_loss: 8.6815 -

In [ ]:
RESNET50_MODEL.save('resnet50_model.h5')

In [ ]:
MOBILENETv2_HISTORY = compile_and_fit(MOBILENETv2_MODEL)

Epoch 1/10
250/250 [==============================] - 97s 218ms/step - loss: 2.4697 - accuracy: 0.4176 - val_loss: 52.6307 - val_accuracy: 0.0950
Epoch 2/10
250/250 [==============================] - 45s 180ms/step - loss: 0.9711 - accuracy: 0.6302 - val_loss: 29.0336 - val_accuracy: 0.0945
Epoch 3/10
250/250 [==============================] - 45s 179ms/step - loss: 0.6287 - accuracy: 0.7600 - val_loss: 11.0969 - val_accuracy: 0.1440
Epoch 4/10
250/250 [==============================] - 45s 179ms/step - loss: 0.4599 - accuracy: 0.8329 - val_loss: 7.6990 - val_accuracy: 0.2220
Epoch 5/10
250/250 [==============================] - 45s 179ms/step - loss: 0.3794 - accuracy: 0.8654 - val_loss: 6.0190 - val_accuracy: 0.2725
Epoch 6/10
250/250 [==============================] - 45s 179ms/step - loss: 0.3570 - accuracy: 0.8740 - val_loss: 7.4211 - val_accuracy: 0.2835
Epoch 7/10
250/250 [==============================] - 45s 181ms/step - loss: 0.4106 - accuracy: 0.8618 - val_loss: 6.0780 - val

In [ ]:
MOBILENETv2_MODEL.save('mobilenetv2_model.h5')